# Тестовое задание

In [15]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string

In [3]:
count_vect = CountVectorizer()
tfidf_vect = TfidfVectorizer()

## Cкачиваем файл с данными

In [4]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [ ]:
df = pd.read_json ('reviews_Electronics_5.json', lines=True )
df.to_csv ('Electronics_5.csv', index = None)

## Предобработка

In [5]:
dat = pd.read_csv("Electronics_5.csv")[["asin","reviewText", "overall"]]

## Добавляем бинрную оценку

In [6]:
dat["bin"] = np.where(dat.overall > 3, 1, 0)

## Отрежем кусок даты

In [7]:
dat_t, dat = dat, dat[0:11]

## Убираем стоп-слова

In [17]:
dat["words"] = dat.reviewText.str.split()
for i in range(len(dat["words"])):
    dat["words"][i] = (" ").join(list(map(lambda x: x.lower().translate(str.maketrans('', '', string.punctuation)),filter(lambda word: word.lower() not in nltk.corpus.stopwords.words('english'), dat["words"][i]))))

/home/denis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/denis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/denis/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

In [18]:
dat_count = count_vect.fit_transform(dat.words).toarray()
dat_tfidf = tfidf_vect.fit_transform(dat.words).toarray()

## Проверим, что получилось

In [19]:
dat

,asin,reviewText,overall,bin,words
0,0528881469,We got this GPS for my husband who is an (OTR)...,5,1,got gps husband otr road trucker impressed shi...
1,0528881469,"I'm a professional OTR truck driver, and I bou...",1,0,im professional otr truck driver bought tnd 70...
2,0528881469,"Well, what can I say. I've had this unit in m...",3,0,well say ive unit truck four days now prior ga...
3,0528881469,"Not going to write a long review, even thought...",2,0,going write long review even thought unit dese...
4,0528881469,I've had mine for a year and here's what we go...,1,0,ive mine year heres got tries route non truck ...
5,0594451647,I am using this with a Nook HD+. It works as d...,5,1,using nook hd works described hd picture samsu...
6,0594451647,The cable is very wobbly and sometimes disconn...,2,0,cable wobbly sometimes disconnects itselfthe p...
7,0594451647,This adaptor is real easy to setup and use rig...,5,1,adaptor real easy setup use right box problem ...
8,0594451647,This adapter easily connects my Nook HD 7&#34;...,4,1,adapter easily connects nook hd 734 hdtv hdmi ...
9,0594451647,This product really works great but I found th...,5,1,product really works great found following ite...


In [20]:
dat_count

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [2, 2, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
dat_tfidf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.0574219 ,
        0.        ],
       [0.05553579, 0.0649721 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08045601, 0.        ,
        0.09412662],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])